## 第一部分：了解 nn.Module的基本操作

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [3]:
model = models.resnet18()

### 打印出 model底下所有 parameters 的 name 以及對應的 shape 

In [36]:
print(repr("name").ljust(35),"|",repr("param.shape").ljust(35),"|",repr("param.requires_grad").ljust(5), "\n", "="*100)
for name, param in model.named_parameters():
    print(repr(name).ljust(35),"|",repr(param.shape).ljust(35),"|",repr(param.requires_grad).ljust(5))

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('Total number of parameters：', params)

'name'                              | 'param.shape'                       | 'param.requires_grad' 
'conv1.weight'                      | torch.Size([64, 3, 7, 7])           | True 
'bn1.weight'                        | torch.Size([64])                    | True 
'bn1.bias'                          | torch.Size([64])                    | True 
'layer1.0.conv1.weight'             | torch.Size([64, 64, 3, 3])          | True 
'layer1.0.bn1.weight'               | torch.Size([64])                    | True 
'layer1.0.bn1.bias'                 | torch.Size([64])                    | True 
'layer1.0.conv2.weight'             | torch.Size([64, 64, 3, 3])          | True 
'layer1.0.bn2.weight'               | torch.Size([64])                    | True 
'layer1.0.bn2.bias'                 | torch.Size([64])                    | True 
'layer1.1.conv1.weight'             | torch.Size([64, 64, 3, 3])          | True 
'layer1.1.bn1.weight'               | torch.Size([64])                    | True 

### 為了使 forward propagation 加速 並降低 memory 使用量，請將所有 parameters 的requires_grad 關閉，關閉之後執行  forward propagation

In [35]:
input_ = torch.randn(1, 3, 128, 128)

In [40]:
# Disable autograd to increase computation speed
for param in model.parameters():
    param.requires_gradd = False

In [41]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])


## 第二部分：依照指令，以較簡潔的方式搭建出模型

* input_shape = torch.Size([10, 12])
* 先經過一層 nn.Linear(12, 10)
* 經過10層 nn.Linear(10, 10)
* 最後經過 nn.Linear(10, 3) 輸出
* 每一個 nn.Linear過完後要先經過 nn.BatchNorm1d 才能到下一層，輸出層不用


In [42]:
input_ = torch.randn(10,12)
## 示範
Linear = nn.Linear(12,10)
BN = nn.BatchNorm1d(10)

In [69]:
from torch.nn.modules.batchnorm import BatchNorm1d
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.sequential = nn.Sequential(
            nn.Linear(12,10),
            nn.BatchNorm1d(10)
        )

        self.repeat_linear = nn.ModuleList([
          nn.Sequential(
            nn.Linear(10,10),
            nn.BatchNorm1d(10)
            ) for i in range(10)                               
        ])
        self.output = nn.Linear(10,3)

    def forward(self, x):
        x = self.sequential(x)
        for layer in self.repeat_linear:
            x = layer(x)
        x = self.output(x)
        return x

In [70]:
model = Model()

In [71]:
model

Model(
  (sequential): Sequential(
    (0): Linear(in_features=12, out_features=10, bias=True)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (repeat_linear): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): Sequential(
      (0): Linear(in_features=10, out_fe

In [72]:
input_ = torch.randn(10,12)
outupt = model(input_)

In [73]:
outupt

tensor([[ 0.1122,  0.4593, -0.4120],
        [ 0.4059, -0.5670,  0.3421],
        [ 0.4347, -0.2296, -0.0568],
        [-0.5233, -0.1516, -1.1628],
        [ 0.3379, -0.9413,  0.0222],
        [ 0.6472,  1.7456, -0.2476],
        [ 0.2184,  0.5078, -0.6275],
        [ 0.4732, -0.2369,  0.6626],
        [ 0.4993, -0.5457,  0.1980],
        [-0.5841,  0.3507, -1.4997]], grad_fn=<AddmmBackward0>)